# Amazon Bedrock AgentCore Policy - MCP Server Target Tutorial

## Overview

This notebook demonstrates how to use **Amazon Bedrock AgentCore Policy** with an **MCP Server Target** deployed to **AgentCore Runtime** with OAuth authentication.

### Learning Objectives

- Deploy MCP Server to AgentCore Runtime with OAuth
- Connect MCP Server to Gateway as a Target
- Apply Cedar policies for fine-grained access control
- Test various Cedar policy patterns

### Architecture

```
                                ┌───────────────────────┐
                                │  Policy Engine        │
                                │  (Cedar Policy)       │
                                └───────────┬───────────┘
                                            │ Connected
                                            ▼
┌─────────────────┐             ┌───────────────────────┐             ┌─────────────────────┐
│   Gateway       │  JWT Token  │  AgentCore Gateway    │  MCP over   │  AgentCore Runtime  │
│   Cognito       │────────────>│  + Cedar Policy       │  HTTP/OAuth │  (MCP Server)       │
│                 │             │                       │────────────>│  (approve_claim)    │
└─────────────────┘             └───────────────────────┘             └─────────────────────┘
                                                                              │
                                                                              │ OAuth
                                                                              ▼
                                                                      ┌─────────────────────┐
                                                                      │  Runtime Cognito    │
                                                                      │  (Credential        │
                                                                      │   Provider)         │
                                                                      └─────────────────────┘
```

### Key Differences from Lambda Target

| Aspect | Lambda Target | MCP Server Target |
|--------|---------------|-------------------|
| Backend | AWS Lambda Function | MCP Server on AgentCore Runtime |
| Protocol | Lambda Invoke | MCP over HTTP with OAuth |
| Tool Discovery | Inline Schema | SynchronizeGatewayTargets API |
| Hosting | AWS Managed Lambda | AgentCore Runtime (Container) |
| Authentication | N/A (internal) | OAuth2 Credential Provider |

---

## Part 1: Environment Setup

Load required libraries and configuration from the deployed MCP Runtime.

### Environment Setup Options

**Option 1: UV Virtual Environment (Recommended)**
```bash
cd ../00_setup
chmod +x create_uv_virtual_env.sh
./create_uv_virtual_env.sh AgentCorePolicyMCP
```
Then select the created kernel in Jupyter.

In [ ]:
import json
import sys
import time
from pathlib import Path

import boto3

# Add parent directory to path for common imports
sys.path.insert(0, str(Path.cwd().parent))

# Import utility functions from common folder
from common.auth_utils import (
    get_bearer_token,
    decode_token,
    make_gateway_request,
    analyze_response,
    display_test_result,
)
from common.gateway_utils import (
    validate_and_fix_gateway_authorizer,
    attach_policy_engine_to_gateway,
    list_gateway_targets,
)
from common.policy_utils import (
    get_policy_engine,
    create_cedar_policy,
    wait_for_policy_active,
    delete_policy,
    cleanup_existing_policies,
    ensure_policy_engine,
)

print("✓ Libraries and utilities loaded")

### Step 1.1: Load Configuration

Load the deployed MCP Runtime configuration from `runtime_config.json` and Gateway Cognito from `gateway_cognito_config.json`.

In [ ]:
# Load Runtime Configuration
runtime_config_path = Path.cwd() / "runtime_config.json"

if not runtime_config_path.exists():
    raise FileNotFoundError(
        "runtime_config.json not found. Please run deploy_mcp_runtime.py first."
    )

with open(runtime_config_path, "r") as f:
    RUNTIME_CONFIG = json.load(f)

# Load Gateway Cognito Configuration
gateway_cognito_path = Path.cwd() / "gateway_cognito_config.json"

if not gateway_cognito_path.exists():
    raise FileNotFoundError(
        "gateway_cognito_config.json not found."
    )

with open(gateway_cognito_path, "r") as f:
    GATEWAY_COGNITO = json.load(f)

print("✓ Configuration files loaded")

In [ ]:
# Extract key configuration values
REGION = RUNTIME_CONFIG["region"]
GATEWAY_URL = RUNTIME_CONFIG["gateway_url"]
GATEWAY_ID = RUNTIME_CONFIG["gateway_id"]
GATEWAY_ARN = RUNTIME_CONFIG["gateway_arn"]
POLICY_ENGINE_ARN = RUNTIME_CONFIG["policy_engine_arn"]
MCP_TARGET_ID = RUNTIME_CONFIG["target_id"]

# Gateway Cognito for API calls
CLIENT_ID = GATEWAY_COGNITO["client_id"]
CLIENT_SECRET = GATEWAY_COGNITO["client_secret"]
TOKEN_ENDPOINT = GATEWAY_COGNITO["token_endpoint"]
SCOPE = GATEWAY_COGNITO["scope"]

# Extract Policy Engine ID from ARN
POLICY_ENGINE_ID = POLICY_ENGINE_ARN.split("/")[-1]

# MCP Target Name (for tool naming)
MCP_TARGET_NAME = "RefundMCPServerTarget"

print("✓ Configuration loaded")
print(f"  REGION: {REGION}")
print(f"  GATEWAY_ID: {GATEWAY_ID}")
print(f"  GATEWAY_URL: {GATEWAY_URL}")
print(f"  GATEWAY_ARN: {GATEWAY_ARN}")
print(f"  POLICY_ENGINE_ID: {POLICY_ENGINE_ID}")
print(f"  MCP_TARGET_ID: {MCP_TARGET_ID}")
print(f"  TOKEN_ENDPOINT: {TOKEN_ENDPOINT}")

### Step 1.2: Initialize AWS Clients

In [ ]:
session = boto3.Session(region_name=REGION)

sts_client = session.client("sts")
policy_client = session.client("bedrock-agentcore-control", region_name=REGION)
gateway_control_client = session.client("bedrock-agentcore-control", region_name=REGION)

# Get Account ID
ACCOUNT_ID = sts_client.get_caller_identity()["Account"]

print("✓ AWS clients initialized")
print(f"  Account ID: {ACCOUNT_ID}")
print(f"  Region: {REGION}")

### Step 1.3: Verify Gateway and Policy Engine Connection

In [ ]:
# Verify Gateway has the MCP target
targets = list_gateway_targets(gateway_control_client, GATEWAY_ID)
print(f"\n✓ Gateway targets: {len(targets)}")
for target in targets:
    print(f"  - {target.get('name', 'N/A')} (ID: {target.get('targetId')})")

In [ ]:
# Verify Policy Engine is attached
engine = get_policy_engine(policy_client, POLICY_ENGINE_ID)
if engine:
    print(f"✓ Policy Engine: {POLICY_ENGINE_ID}")
    print(f"  Status: {engine.get('status', 'N/A')}")
else:
    print("✗ Policy Engine not found")

---

## Part 2: Understanding MCP Target Tool Names

When an MCP Server is connected as a Gateway Target, tool names follow this format:

```
{TargetName}___{tool_name}
```

For our MCP Server with tools `refund`, `get_order`, and `approve_claim`, the Gateway tool names are:

| MCP Server Tool | Gateway Tool Name |
|-----------------|-------------------|
| `refund` | `RefundMCPServerTarget___refund` |
| `get_order` | `RefundMCPServerTarget___get_order` |
| `approve_claim` | `RefundMCPServerTarget___approve_claim` |

In Cedar policies, we reference these as:

```cedar
action == AgentCore::Action::"RefundMCPServerTarget___approve_claim"
```

In [ ]:
# Define tool names for testing
# Note: We use approve_claim for testing as it works reliably with Cedar policies
TOOL_NAME = f"{MCP_TARGET_NAME}___approve_claim"

print(f"✓ Tool name for Cedar policy: {TOOL_NAME}")

---

## Part 3: Test Scenario 1 - String Equality

Create a Cedar policy that only allows claims with `risk_level == "low"`.

### Cedar Policy Pattern

Use `context.input.{field}` to access tool input parameters:

```cedar
permit(principal, action, resource)
when {
    context.input.risk_level == "low"
};
```

### Step 3.0: Clean Up Existing Policies

In [ ]:
# Clean up existing policies for fresh start
cleanup_existing_policies(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    require_confirmation=False  # Auto delete
)

# Track created policies for cleanup
CREATED_POLICIES = []

### Step 3.1: Create String Equality Policy

In [ ]:
print("=" * 70)
print("Test Scenario 1: String Equality")
print("=" * 70)

policy_name = f"string_equality_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"{TOOL_NAME}",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    context.input has risk_level &&
    context.input.risk_level == "low"
}};'''

print("\nCedar Policy:")
print("-" * 60)
print(cedar_statement)
print("-" * 60)
print("\nThis policy allows claims only when risk_level == 'low'")

In [ ]:
# Create the policy
policy_id = create_cedar_policy(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="Allow only low risk claims"
)

if policy_id:
    CREATED_POLICIES.append(policy_id)
    
    print("\n⏳ Waiting for policy to become ACTIVE...")
    if wait_for_policy_active(policy_client, POLICY_ENGINE_ID, policy_id):
        print("✓ Policy is ACTIVE")
    else:
        print("⚠️  Policy did not become ACTIVE")
else:
    print("\n✗ Policy creation failed")

### Step 3.2: Get Bearer Token

In [ ]:
token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    scope=SCOPE
)

print(f"✓ Token received")
print(f"  Token (first 50 chars): {token[:50]}...")

claims = decode_token(token)
print(f"\n🔍 Token Claims:")
print(f"  client_id: {claims.get('client_id', 'N/A')}")
print(f"  scope: {claims.get('scope', 'N/A')}")

### Step 3.3: Test - risk_level="low" (Expected: ALLOWED)

In [ ]:
print("\n" + "=" * 70)
print("Test 1.1: risk_level='low' request")
print("=" * 70)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-001", "amount": 500, "risk_level": "low"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='low')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "risk_level='low' should be ALLOWED")

### Step 3.4: Test - risk_level="high" (Expected: DENIED)

In [ ]:
print("\n" + "=" * 70)
print("Test 1.2: risk_level='high' request")
print("=" * 70)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-002", "amount": 500, "risk_level": "high"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='high')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "risk_level='high' should be DENIED")

---

## Part 4: Test Scenario 2 - Pattern Matching (like)

Create a Cedar policy that allows claims with risk_level containing "low".

### Cedar Policy Pattern

| Pattern | Matches |
|---------|--------|
| `"*low*"` | Contains "low" |
| `"low*"` | Starts with "low" |
| `"*low"` | Ends with "low" |

```cedar
permit(principal, action, resource)
when {
    context.input.risk_level like "*low*"
};
```

In [ ]:
# Clean up previous policy
print("=" * 70)
print("Test Scenario 2: Pattern Matching (like)")
print("=" * 70)

print("\nCleaning up previous policies...")
for pid in CREATED_POLICIES:
    delete_policy(policy_client, POLICY_ENGINE_ID, pid)
CREATED_POLICIES.clear()

### Step 4.1: Create Pattern Matching Policy

In [ ]:
policy_name = f"pattern_matching_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"{TOOL_NAME}",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    context.input has risk_level &&
    context.input.risk_level like "*low*"
}};'''

print("\nCedar Policy:")
print("-" * 60)
print(cedar_statement)
print("-" * 60)
print("\nThis policy allows claims when risk_level contains 'low'")
print("  ✓ Matches: 'low', 'very_low', 'low_priority'")
print("  ✗ No match: 'high', 'medium', 'critical'")

In [ ]:
# Create the policy
policy_id = create_cedar_policy(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="Allow claims with risk_level containing 'low'"
)

if policy_id:
    CREATED_POLICIES.append(policy_id)
    
    print("\n⏳ Waiting for policy to become ACTIVE...")
    if wait_for_policy_active(policy_client, POLICY_ENGINE_ID, policy_id):
        print("✓ Policy is ACTIVE")

### Step 4.2: Test - risk_level="very_low" (Expected: ALLOWED)

In [ ]:
print("\n" + "=" * 70)
print("Test 2.1: risk_level='very_low' (contains 'low')")
print("=" * 70)

# Get fresh token
token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    scope=SCOPE
)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-003", "amount": 500, "risk_level": "very_low"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='very_low')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "'very_low' contains 'low' - should be ALLOWED")

### Step 4.3: Test - risk_level="high" (Expected: DENIED)

In [ ]:
print("\n" + "=" * 70)
print("Test 2.2: risk_level='high' (does not contain 'low')")
print("=" * 70)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-004", "amount": 500, "risk_level": "high"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='high')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "'high' does not contain 'low' - should be DENIED")

---

## Part 5: Test Scenario 3 - OR Conditions

Create a Cedar policy that allows claims with risk_level "low" OR "medium".

### Cedar Policy Pattern

```cedar
permit(principal, action, resource)
when {
    context.input.risk_level == "low" ||
    context.input.risk_level == "medium"
};
```

In [ ]:
# Clean up previous policy
print("=" * 70)
print("Test Scenario 3: OR Conditions")
print("=" * 70)

print("\nCleaning up previous policies...")
for pid in CREATED_POLICIES:
    delete_policy(policy_client, POLICY_ENGINE_ID, pid)
CREATED_POLICIES.clear()

### Step 5.1: Create OR Condition Policy

In [ ]:
policy_name = f"or_condition_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"{TOOL_NAME}",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    context.input has risk_level &&
    (context.input.risk_level == "low" || context.input.risk_level == "medium")
}};'''

print("\nCedar Policy:")
print("-" * 60)
print(cedar_statement)
print("-" * 60)
print("\nThis policy allows claims when risk_level is 'low' OR 'medium'")
print("  ✓ Allowed: 'low', 'medium'")
print("  ✗ Denied: 'high', 'critical'")

In [ ]:
# Create the policy
policy_id = create_cedar_policy(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="Allow claims with low or medium risk"
)

if policy_id:
    CREATED_POLICIES.append(policy_id)
    
    print("\n⏳ Waiting for policy to become ACTIVE...")
    if wait_for_policy_active(policy_client, POLICY_ENGINE_ID, policy_id):
        print("✓ Policy is ACTIVE")

### Step 5.2: Test - risk_level="medium" (Expected: ALLOWED)

In [ ]:
print("\n" + "=" * 70)
print("Test 3.1: risk_level='medium'")
print("=" * 70)

# Get fresh token
token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    scope=SCOPE
)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-005", "amount": 500, "risk_level": "medium"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='medium')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "'medium' matches OR condition - should be ALLOWED")

### Step 5.3: Test - risk_level="high" (Expected: DENIED)

In [ ]:
print("\n" + "=" * 70)
print("Test 3.2: risk_level='high'")
print("=" * 70)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-006", "amount": 500, "risk_level": "high"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='high')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "'high' does not match OR condition - should be DENIED")

---

## Part 6: Test Scenario 4 - Negation (NOT)

Create a Cedar policy that allows claims when risk_level is NOT "critical".

### Cedar Policy Pattern

```cedar
permit(principal, action, resource)
when {
    !(context.input.risk_level == "critical")
};
```

In [ ]:
# Clean up previous policy
print("=" * 70)
print("Test Scenario 4: Negation (NOT)")
print("=" * 70)

print("\nCleaning up previous policies...")
for pid in CREATED_POLICIES:
    delete_policy(policy_client, POLICY_ENGINE_ID, pid)
CREATED_POLICIES.clear()

### Step 6.1: Create Negation Policy

In [ ]:
policy_name = f"negation_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"{TOOL_NAME}",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    !(context.input has risk_level && context.input.risk_level == "critical")
}};'''

print("\nCedar Policy:")
print("-" * 60)
print(cedar_statement)
print("-" * 60)
print("\nThis policy allows claims when risk_level is NOT 'critical'")
print("  ✓ Allowed: 'low', 'medium', 'high'")
print("  ✗ Denied: 'critical'")

In [ ]:
# Create the policy
policy_id = create_cedar_policy(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="Allow all claims except critical risk"
)

if policy_id:
    CREATED_POLICIES.append(policy_id)
    
    print("\n⏳ Waiting for policy to become ACTIVE...")
    if wait_for_policy_active(policy_client, POLICY_ENGINE_ID, policy_id):
        print("✓ Policy is ACTIVE")

### Step 6.2: Test - risk_level="high" (Expected: ALLOWED)

In [ ]:
print("\n" + "=" * 70)
print("Test 4.1: risk_level='high' (not critical)")
print("=" * 70)

# Get fresh token
token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    scope=SCOPE
)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-007", "amount": 500, "risk_level": "high"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='high')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "'high' is not 'critical' - should be ALLOWED")

### Step 6.3: Test - risk_level="critical" (Expected: DENIED)

In [ ]:
print("\n" + "=" * 70)
print("Test 4.2: risk_level='critical'")
print("=" * 70)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-008", "amount": 500, "risk_level": "critical"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='critical')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "'critical' matches negation - should be DENIED")

---

## Part 7: Advanced Patterns

### Combined Conditions

You can combine multiple conditions to create complex access control scenarios.

In [ ]:
# Example: Combined conditions
print("\nAdvanced Pattern: Combined Conditions")
print("=" * 70)

combined_cedar = f'''permit(principal,
    action == AgentCore::Action::"{TOOL_NAME}",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    context.input has risk_level &&
    (context.input.risk_level == "low" || context.input.risk_level == "medium") &&
    !(context.input has claim_id && context.input.claim_id like "*FRAUD*")
}};'''

print("Combined Cedar Policy:")
print("-" * 60)
print(combined_cedar)
print("-" * 60)
print("\nThis policy allows when:")
print("  ✓ risk_level is 'low' OR 'medium'")
print("  AND")
print("  ✓ claim_id does NOT contain 'FRAUD'")

### Pattern Matching Variations

| Pattern | Matches | Example |
|---------|---------|--------|
| `"*admin*"` | Contains "admin" | "admin", "super_admin", "admin_user" |
| `"admin*"` | Starts with "admin" | "admin", "admin_user" |
| `"*admin"` | Ends with "admin" | "admin", "super_admin" |
| `"team-*"` | Starts with "team-" | "team-alpha", "team-beta" |

In [ ]:
# Example: Wildcard pattern for claim_id
print("\nAdvanced Pattern: Wildcard for Claim ID Prefix")
print("=" * 70)

prefix_cedar = f'''permit(principal,
    action == AgentCore::Action::"{TOOL_NAME}",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    context.input has claim_id &&
    context.input.claim_id like "AUTO-*"
}};'''

print("Pattern Matching Cedar Policy:")
print("-" * 60)
print(prefix_cedar)
print("-" * 60)
print("\nThis policy allows:")
print("  ✓ claim_id: 'AUTO-001', 'AUTO-CLAIM-123'")
print("  ✗ claim_id: 'MANUAL-001', 'CLM-AUTO-001'")

---

## Part 8: Cleanup

Delete policies created during testing.

In [ ]:
print("=" * 70)
print("Cleanup")
print("=" * 70)

print(f"\nDeleting {len(CREATED_POLICIES)} policies...")
for pid in CREATED_POLICIES:
    delete_policy(policy_client, POLICY_ENGINE_ID, pid)

CREATED_POLICIES.clear()
print("\n✓ Cleanup complete")

---

## Conclusion

This tutorial demonstrated Cedar policy enforcement on MCP Server Targets. You learned:

✅ Deploy MCP Server to AgentCore Runtime with OAuth authentication  
✅ Connect MCP Server as Gateway Target  
✅ Apply Cedar policies using `context.input.{field}` for tool arguments  
✅ String equality checks (`context.input.risk_level == "low"`)  
✅ Pattern matching with `like` operator (`context.input.risk_level like "*low*"`)  
✅ OR conditions (`risk_level == "low" || risk_level == "medium"`)  
✅ Negation (`!(context.input.risk_level == "critical")`)  

### Key Cedar Syntax Patterns

| Condition Type | Cedar Syntax | Example |
|----------------|--------------|--------|
| String Equality | `context.input.field == "value"` | `risk_level == "low"` |
| Pattern Match | `context.input.field like "*pattern*"` | `risk_level like "*low*"` |
| OR Condition | `condition1 \|\| condition2` | `risk_level == "low" \|\| risk_level == "medium"` |
| Negation | `!(condition)` | `!(risk_level == "critical")` |
| Field Existence | `context.input has field` | `context.input has risk_level` |

### Tool Name Format

```
{TargetName}___{tool_name}
```

Example: `RefundMCPServerTarget___approve_claim`

### Next Steps

1. **Production Deployment** - Apply these patterns to your production MCP servers
2. **JWT Claims** - Combine with `principal.getTag()` for identity-based access control
3. **Monitoring** - Set up CloudWatch alarms for policy denials